In [1]:
import pandas as pd
import numpy as np


In [64]:
nat_gas = pd.read_csv('./Nat_Gas.csv')
nat_gas['Dates'] = pd.to_datetime(nat_gas['Dates'])
nat_gas['Dates'].head()
df = nat_gas.copy()
df.index = df['Dates']
del df['Dates']
df.head()

/var/folders/h0/722z94dd3fb0pfv4wg3qmdkh0000gn/T/ipykernel_76537/2926170649.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  nat_gas['Dates'] = pd.to_datetime(nat_gas['Dates'])


,Prices
Dates,
2020-10-31,10.1
2020-11-30,10.3
2020-12-31,11.0
2021-01-31,10.9
2021-02-28,10.9


# Task
- create a prototype **contract pricing** model 
    - write a function that is able to use the data from Task 1. 

In [65]:
df.head()

,Prices
Dates,
2020-10-31,10.1
2020-11-30,10.3
2020-12-31,11.0
2021-01-31,10.9
2021-02-28,10.9


In [17]:
# We want to retain the date index from the last task, so add parse_dates=True
prices = pd.read_csv('./01-exported_pred_prices_nat_gas.csv', index_col=0, parse_dates=True)
prices.head()

,Prices
2020-10-31,10.100000
2020-11-01,10.106667
2020-11-02,10.113333
2020-11-03,10.120000
2020-11-04,10.126667


In [45]:
float(prices.loc['2020-10-31'])

/var/folders/h0/722z94dd3fb0pfv4wg3qmdkh0000gn/T/ipykernel_76537/3697812350.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(prices.loc['2020-10-31'])


10.1

In [116]:
def calculate_margin(volume_list):

    '''
        volume_list will have to be written in the following manner:
            volume_list = [(date1, 100), (date2, -400), (date3, 50), (date4, 75), (date5, -175)]
        wherein the negative implies it is a withdrawal, and positive stands for injection
    '''

    # Calculate storage costs
    earliest_date = min(pd.to_datetime(date) for date, _ in volume_list)
    latest_date = max(pd.to_datetime(date) for date, _ in volume_list)
    storage_duration = (latest_date - earliest_date).days / 30  # Months
    storage_cost = storage_duration * 100000  # $100K/month

    # Calculate injection and withdrawal costs, and revenue
    total_operations = len(volume_list) # Assuming that we can only inject OR withdraw and not both at a given date
    transport_and_operator_costs = 0
    revenue = 0
    for date, volume in volume_list:
        transport_cost = abs(volume) * 10000  # $10K/MMBtu (use absolute value for transport cost)
        transport_and_operator_costs += transport_cost + 50000  # Add flat operator fee

        date =  pd.to_datetime(date) # Enables flexibility in how dates are entered
        price = prices.loc[date, 'Prices']  # Get price for either injection or withdrawal from our price df
        

        if volume > 0:  # Injection (buying)
            revenue -= volume * price * 1000000  # Subtract cost of buying
        else:  # Withdrawal (selling)
            revenue += -volume * price * 1000000  # Add revenue from selling

    # Calculate margin
    margin = revenue - storage_cost - transport_and_operator_costs

    return format(revenue, ".2f")

In [117]:
volume_list = [
    ('June 29, 2021', 3), #injects 30MMBtu on June 6th, 2021, i.e., bought 
    ('August 15, 2021', -1.5), # sells 5 on Aug 15, 2021, i.e., withdrew 5MMBtu
    ('Sep 12, 2021', 1),
    ('Dec 29, 2021', -2.5)
]

In [118]:
calculate_margin(volume_list)

'3518903.23'